## Libraries:

Below the libraries usedin the analysis. If you get an error while importing a library just ru the following command in your notebook: !pip install packgename

In [5]:
!pip install docx2txt
!pip install pyemd
!pip install langid
!pip install pyemd
!pip install inflect

In [6]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import langid
import docx2txt

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data loading: 
This cell allow me to know the subfolders of my input since I'm working on kaggle kernel.

In [7]:
import os
print(os.listdir("../input"))

['googlenewsvectorsnegative300', 'mydata']


Update the path of the files while loading them. 

In [8]:
#Reading the test cases 
Data = pd.read_csv("../input/mydata/1000_case_text.csv")

#Reading the word documents
doc1 = docx2txt.process("../input/mydata/Document_1.docx")
doc2 = docx2txt.process("../input/mydata/Document_2.docx")

## Data Cleaning:

I defined two cleaning fuctions: one for word documents and one for test cases. 

In [9]:
#Preprocessing function: 

contraction = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                 "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                 "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                  "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                  "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                  "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                  "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                  "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                   "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                   "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                   "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                   "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                   "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                   "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                   "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                   "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                    "you're": "you are", "you've": "you have"}

stop_words = set(stopwords.words('english'))



def textprocessing(text): 
    Text = text.lower() # Convert everything to lowercase
    Text = BeautifulSoup(Text, "lxml").text #Remove HTML tags: Extract text from tags 
    Text = re.sub(r'\([^)]*\)', '', Text) # Remove any text inside the parenthesis
    Text = re.sub('"','', Text) 
    Text = ' '.join([contraction[t] if t in contraction else t for t in Text.split(" ")])    
    Text = re.sub(r"'s\b","",Text)  #Remove (‘s) \b: Returns a match where the specified characters are at the beginning or at the end of a word
    Text = re.sub("[^a-zA-Z]", " ", Text)
    tokens = [w for w in Text.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

### A -  Word documents preprocessing:

Ones we load the data, we feed it into a processing part which consists of several steps:
We define a processing function that includes all the following sub functions:

 - Convert the reviews and summaries to lowercase
 - Remove HTML tags
 - Contraction mapping: that consists of importing a contraction dictionary for example: did'nt ~ did not
 - Remove (‘s)
 - Remove any text inside the parenthesis ( )
 - Eliminate punctuations and special characters
 - Eliminate stop words
 - Eliminate short words

In [10]:
CleanedW1 = textprocessing(doc1)
CleanedW2 = textprocessing(doc2)

### B - Test cases preprocessing:

The test cases processing includes the following functions: 

* removing non asci words
* replacing numbers with strings: 2 becomes two...
* Setting the whole text into lowercase
* removing punctuation
* remove stopwords like articles...

In [11]:
import re, string, unicodedata
import inflect
import nltk
from bs4 import BeautifulSoup
from nltk import tokenize
from nltk.tokenize import word_tokenize


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    remove_non_ascii(words)
    words = replace_numbers(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return ' '.join(word for word in words)

### Test cases tokenization: 

The texts are written with different languages and since the packages used for text processing and modeling include language in their functions we must work with each set of texts seperatly ( by language). Therefore, we used the langid function to verify the language. 

In [12]:
#We get rows where the variable 'CNSLD_EXTRN_CMNT' in not NA
DATA  = Data[Data['CNSLD_EXTRN_CMNT'].notna()]

In [13]:
EnData = pd.DataFrame()

In [14]:
v = DATA.shape[0]
a=0
for i in range(v):
    if langid.classify(DATA.iloc[i][3])[0] == 'en':
        EnData = EnData.append(DATA.iloc[i])
        a=a+i

#### Tokenization:
It refers to splitting the text into words since the normalize fucntion takes as input list of words and not text. 

In [15]:
TokensTests = [word_tokenize(i) for i in EnData['CNSLD_EXTRN_CMNT']]

In [16]:
CleanedTEXT = []
for t in TokensTests:
    CleanedTEXT.append(normalize(t))

In [17]:
EnData['CleanedText']=CleanedTEXT

The 'EnData' data frame contains the cleaned text tat ill be fed into our model. 

In [18]:
EnData.head()

,CASE_ID,CASE_SMRY,CNSLD_EXTRN_CMNT,CNSLD_INTRN_CMNT,DEL_IND,EDL_LOAD_TS,CleanedText
0,DTAC-836068,JHB // hydraulic surge scv,Lewis:We are seeing normal here.The charge der...,NaN,0.0,1.583593e+09,lewis seeing normal herethe charge derate soft...
1,DTAC-836134,523628.10 (def dosing pressure abnormal) /ques...,"Ben,Reference attached Solution 117649, John D...",NaN,0.0,1.583593e+09,ben reference attached solution one hundred an...
2,DTAC-836152,as code 629.13,DTAC: Called and Spoke with Harold. The softwa...,NaN,0.0,1.583593e+09,dtac called spoke harold software updated prog...
3,DTAC-836198,"Bad ecu,need payload information,",DTAC: Hello Edward. If You are getting error ...,NaN,0.0,1.583593e+09,dtac hello edward getting error 1057037 please...
4,DTAC-83635,Failed fuel sensor,thanks feedback case to us On this tractor fue...,None,0.0,1.583593e+09,thanks feedback case us tractor fuel level sho...


## Modeling: 
We will quantify the meaning similarity between the test cases and the word documents using pretrained word2vec model. 
In this context, we worked with pre-trained word embedding model 'Word2vec' trained on Google News to get the vector representations for each text's word either in the test file and word doc one. Therefore, we are facing a problem of similarity computing presented as follows.

Suppose we want to calculate the similarity between those two sentences:

𝑆1  = 'obama speaks media illinois' ,  𝑆2  = 'president greets press chicago'
It is true that those sentences don't share the same words but maybe they conveyed the same information. At this stage, each sentence is represented as a matrix but the question that arises how could you make a match between the similar words belonging to each sentence. For example, How could you associate Obama to president?

Consequently, we use Word Moving Distance optimization algorithm inspired from the usual optimization problem Earth Mover Distance. It allows transfer every word from sentence 1 to sentence 2 because algorithm does not know “obama” should transfer to “president”. At the end it will choose the minimum transportation cost to transport every word from sentence 1 to sentence 2.

By analogy, we will aplly it in our case study by considering S1 : test case and S2: word doc. 


First, we load pretrained word2vec word embedding using gensim library

In [19]:
import gensim

In [20]:
model = gensim.models.KeyedVectors.load_word2vec_format('../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)

In [21]:
CleanedW1

'solution number solution summary active implement guidance igrade connect mobile planting surface water pro function using universal receiver tractor integrated receiver publication date mar paper copies solutions may current solutions complaint symptom igrade max cut feature function connected integrated receiver active implement guidance function connected integrated receiver connect mobile function connected integrated receiver surface water pro function connected integrated receiver problem situation connected integrated receiver using universal receiver app controller looking wrong source address receiver information integrated receiver claim source address moves universal receiver following normal iso address claiming rules app controller looks source address receiver information connected integrated receiver using universal receiver connect mobile wireless data server look integrated receiver located source however receiver stops transmitting data bus since universal starfire a

## Exmample of distance computing
We feed into wmd function two sentences similar in meaning. More similar they are, the distance converged t 0.

In [25]:
#model.init_sims(replace=True)
S1= 'beauty of nature'
S2 ='her beauty is natural'
model.wmdistance(S1, S2)

0.752898828221336

In [26]:
#model.init_sims(replace=True)
model.wmdistance(EnData['CleanedText'][0], CleanedW2)

0.3769468686540851

In [27]:
EnData.head()

,CASE_ID,CASE_SMRY,CNSLD_EXTRN_CMNT,CNSLD_INTRN_CMNT,DEL_IND,EDL_LOAD_TS,CleanedText
0,DTAC-836068,JHB // hydraulic surge scv,Lewis:We are seeing normal here.The charge der...,NaN,0.0,1.583593e+09,lewis seeing normal herethe charge derate soft...
1,DTAC-836134,523628.10 (def dosing pressure abnormal) /ques...,"Ben,Reference attached Solution 117649, John D...",NaN,0.0,1.583593e+09,ben reference attached solution one hundred an...
2,DTAC-836152,as code 629.13,DTAC: Called and Spoke with Harold. The softwa...,NaN,0.0,1.583593e+09,dtac called spoke harold software updated prog...
3,DTAC-836198,"Bad ecu,need payload information,",DTAC: Hello Edward. If You are getting error ...,NaN,0.0,1.583593e+09,dtac hello edward getting error 1057037 please...
4,DTAC-83635,Failed fuel sensor,thanks feedback case to us On this tractor fue...,None,0.0,1.583593e+09,thanks feedback case us tractor fuel level sho...


We compute the similarity between word document1 (Word document2) and all the test cases. They are represented by 'SimW1' and 'SimW2' in the 'EnData' dataframe

In [28]:
N=EnData.shape[0]
SimEvaluation1 =[]
SimEvaluation2=[]
#model.init_sims(replace=True)
for i in range(N): 
    SimEvaluation1.append(model.wmdistance(EnData.iloc[i]['CleanedText'], CleanedW1))
    SimEvaluation2.append(model.wmdistance(EnData.iloc[i]['CleanedText'], CleanedW2))

EnData['SimW1']=SimEvaluation1
EnData['SimW2'] = SimEvaluation2

In [29]:
EnData.head()

,CASE_ID,CASE_SMRY,CNSLD_EXTRN_CMNT,CNSLD_INTRN_CMNT,DEL_IND,EDL_LOAD_TS,CleanedText,SimW1,SimW2
0,DTAC-836068,JHB // hydraulic surge scv,Lewis:We are seeing normal here.The charge der...,NaN,0.0,1.583593e+09,lewis seeing normal herethe charge derate soft...,0.445538,0.376947
1,DTAC-836134,523628.10 (def dosing pressure abnormal) /ques...,"Ben,Reference attached Solution 117649, John D...",NaN,0.0,1.583593e+09,ben reference attached solution one hundred an...,0.390369,0.410178
2,DTAC-836152,as code 629.13,DTAC: Called and Spoke with Harold. The softwa...,NaN,0.0,1.583593e+09,dtac called spoke harold software updated prog...,0.441620,0.496341
3,DTAC-836198,"Bad ecu,need payload information,",DTAC: Hello Edward. If You are getting error ...,NaN,0.0,1.583593e+09,dtac hello edward getting error 1057037 please...,0.394192,0.428796
4,DTAC-83635,Failed fuel sensor,thanks feedback case to us On this tractor fue...,None,0.0,1.583593e+09,thanks feedback case us tractor fuel level sho...,0.600068,0.614462


In your case study, you're searching for the documens that are close to the word documents. Therefore, the most relevant documents are those with a small distance value.

SortedData1 (SortedData2) represents the data frame that ranks the tests cases from the most similar Word Document1 (Word Document2)) to the less.

In [0]:
SortedData1 =  EnData.sort_values(by=['SimW1'])
SortedData2 =  EnData.sort_values(by=['SimW2'])

In [0]:
SortedData1.to_csv(r'Word1testcases.csv', index = False)
SortedData2.to_csv(r'Word2testcases.csv', index = False)